<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Mnist%20Persian/Mnist_Persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import wandb

In [ ]:
wandb.init(project="PersianMnist", entity="ma_heravi")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [ ]:
# hyperparameters
latent_size = 10
disc_inp_sz = 28*28
img_size = 28
epochs = 10
batch_size = 32
lr = 0.001
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size
}

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_dims):
        super(MyModel, self).__init__()
        self.conv2d = nn.Sequential(
            nn.Conv2d(3, 32, (3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 32, (3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, (3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Flatten(start_dim = 1)
            
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax(),
        )
        
    def forward(self, input_t):
      x = self.conv2d(input_t)
      # print(x.shape)
      return self.fc(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyModel(disc_inp_sz).to(device)

model = model.to(device)
model.train(True)

MyModel(
  (conv2d): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=10, b

In [ ]:
def calc_acc(preds, labels):
    _, preds_max = torch.max(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((28, 28)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/MNIST_persian", transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
 # compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
# train

for epoch in range(1, epochs+1):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")
    wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 1, Loss: 2.0307962894439697, Acc: 0.4202302631578947
Epoch: 2, Loss: 1.849237322807312, Acc: 0.6101973684210525
Epoch: 3, Loss: 1.7057019472122192, Acc: 0.7541118421052632
Epoch: 4, Loss: 1.6880916357040405, Acc: 0.7730263157894737
Epoch: 5, Loss: 1.686398983001709, Acc: 0.7738486842105263
Epoch: 6, Loss: 1.6531925201416016, Acc: 0.8083881578947368
Epoch: 7, Loss: 1.551557183265686, Acc: 0.9177631578947368
Epoch: 8, Loss: 1.5164029598236084, Acc: 0.9457236842105262
Epoch: 9, Loss: 1.5002400875091553, Acc: 0.9638157894736842
Epoch: 10, Loss: 1.4957524538040161, Acc: 0.9654605263157894


In [ ]:
# save
torch.save(model.state_dict(), "PersianMnistFinal.pth")

In [ ]:
# inference

import cv2
import numpy as np


# model.train(False)
model.eval()

# preprocess
img = cv2.imread("3.jpg")
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
preds = model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output